In [ ]:
import pandas as pd
import time

In [ ]:
all_features = ["tweet_type", "Language", "tweet_timestamp", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_timestamp"]

train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 23], sep="\x01")
train['like_bool'] = train.like_timestamp.fillna(0)
train.loc[train.like_bool != 0.0, 'like_bool'] = 1.0
train = train[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]

In [ ]:
def bool_to_str(value):
    "value should be a bool"
    return 'Yes' if value else 'No'

In [ ]:
def construct_line(row, columns, dtypes, string_dict, file_obj, index, time1):
    
    if index % 100000 == 0:
        print(str(time.time() - time1) +": "+ str(index))
        
    global categorical_index
    numerical_index = 0
    new_line = str(row[-1])+ " "
    for i in range(len(columns) - 1):
        value_at_i = row[i]
        dtype_at_i = dtypes[i]
        column_name_at_i = columns[i]
        
        if ((dtype_at_i == object) or (dtype_at_i == bool)): # Categorical Features
            if (dtype_at_i == bool):
                value_at_i =  bool_to_str(value_at_i)
            
            value_to_be_found = column_name_at_i +"_"+ value_at_i
            
            if value_to_be_found in string_dict:
                indexed_value = string_dict[value_to_be_found]
                new_line = new_line + str(indexed_value)+":1 "
            
            else:
                indexed_value = categorical_index
                categorical_index = categorical_index + 1
                new_line = new_line + str(indexed_value)+":1 "
                string_dict[value_to_be_found] = indexed_value
                
        else: # Numerical Features
            new_line = new_line + str(numerical_index) + ":" +str(value_at_i)+" "
            numerical_index = numerical_index + 1
    file_obj.write(new_line+"\n")

### Read Test Set

In [ ]:
all_features = ["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_timestamp"]

test = pd.read_csv("s3://recsys-challenge-2020/val_26_04_2020.tsv", encoding="utf-8",
                    names = all_features, usecols= [6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 19, 23], sep="\x01")
test['like_bool'] = test.like_timestamp.fillna(0)
test.loc[test.like_bool != 0.0, 'like_bool'] = 1.0
test = test[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]

### Write Train and Test Simultaneously

In [ ]:
cnames_numerical = list(train.select_dtypes(exclude=['object', 'bool']).columns)
categorical_index = len(cnames_numerical) - 1 # Categorical indices start from here.
string_dict = {}
f_train = open("f1/like/train.csv.libfm", 'a')
print(len(train))
time1 = time.time()
train.apply(lambda x : construct_line(x, train.columns, train.dtypes, string_dict, f_train, x.name,\
                                         time1), axis = 1)
f_train.close()
print(len(test))
f_test = open("f1/like/test.csv.libfm", 'a')
test.apply(lambda x : construct_line(x, test.columns, test.dtypes, string_dict, f_test, x.name,\
                                       time1), axis = 1)
f_test.close()

In [ ]:
string_dict

### Print Statements

In [ ]:
train_set.dtypes

In [ ]:
val_set.dtypes

In [ ]:
train_set.head()

In [ ]:
val_set.head()

In [ ]:
def construct_line_beta(row, columns, dtypes, string_dict, categorical_index):
    numerical_index = 0
    new_line = str(row[-1])+ " "
    for i in range(len(columns) - 1):
        if ((dtypes[i] == object) or (dtypes[i] == bool)):
            if (dtypes[i] == bool):
                row[i] =  bool_to_str(row[i])
            values = string_dict[columns[i]]
            if row[i] in [x[0] for x in values]:
                index = [x[0] for x in values].index(row[i])
                value_index = values[index][1]
                new_line = new_line + str(value_index)+":1 "
            else:
                value_index = sum(len(v) for v in string_dict.values()) + categorical_index
                new_line = new_line + str(value_index)+":1 "
                values.append((row[i], value_index))
                string_dict[columns[i]] = values
        else:
            new_line = new_line + str(numerical_index) + ":" +str(row[i])+" "
            numerical_index = numerical_index + 1
    print(new_line)

### For Stackoverflow